# Example Notebook For Using Optimizer Tools

In [1]:
import pandas as pd
import datetime as dt

from fps_dfs_optimizer.src.draftkings_tools import get_contests, get_players, EntriesHandler, get_today
from fps_dfs_optimizer.src.projections import get_projections
from fps_dfs_optimizer.src.optimizer import Exposures, LineupOptimizer
from fps_dfs_optimizer.src.generator import LineupGenerator

## Generate today's players sheet
This block will retrieve the players DataFrame for the current date.
Any projections script can be implemented in get_projections(), but right now dummy projections are given

In [2]:
today = get_today()
today = dt.date(2020, 9, 19)
df_contests = get_contests(today)
group_id = df_contests.draft_group_id.unique()[0]
df_players = get_players(group_id)
points, std = get_projections(df_players)
df_players['projections'] = points
df_players['std'] = std

In [3]:
df_players

,Name,ID,Position,Status,Salary,Game,Time,TeamAbbrev,projections,std
0,LeBron James,15432432,PG/SF,Recent,17400,DEN @ LAL,2020-09-19T01:00:00.0000000Z,LAL,101.418278,29.753877
1,Nikola Jokic,15432433,C,Recent,16500,DEN @ LAL,2020-09-19T01:00:00.0000000Z,DEN,84.768098,40.811629
2,Anthony Davis,15432434,PF/C,Breaking,16200,DEN @ LAL,2020-09-19T01:00:00.0000000Z,LAL,67.536264,32.078738
3,Jamal Murray,15432435,PG/SG,Recent,15000,DEN @ LAL,2020-09-19T01:00:00.0000000Z,DEN,59.482523,33.648743
4,Will Barton,15432436,PG,None,13800,DEN @ LAL,2020-09-19T01:00:00.0000000Z,DEN,77.243669,36.438244
...,...,...,...,...,...,...,...,...,...,...
65,Avery Bradley,15432427,PG,None,1000,DEN @ LAL,2020-09-19T01:00:00.0000000Z,LAL,5.754372,2.844129
66,Troy Daniels,15432428,SG,None,1000,DEN @ LAL,2020-09-19T01:00:00.0000000Z,DEN,4.995945,1.834112
67,Quinn Cook,15432429,PG,None,1000,DEN @ LAL,2020-09-19T01:00:00.0000000Z,LAL,5.361482,2.382547
68,Jared Dudley,15432430,PF,None,1000,DEN @ LAL,2020-09-19T01:00:00.0000000Z,LAL,4.439659,2.639839


## Now exposures must be inputted
Here the dataframe above is saved to the data/temp folder for exposures to be filled out.
Once the exposures are filled out, run exp.read_exposures() to pull the sheet into the dataframe.

In [4]:
exp = Exposures(df_players)

Please fill out exposures in ../data/temp/20200917.csv


In [5]:
df_players = exp.read_exposures()

In [6]:
cols = df_players.columns
df_players

,Name,ID,Position,Salary,Status,TeamAbbrev,Time,projections,std,min_exp,max_exp
0,LeBron James,15432432,PG/SF,17400,Recent,LAL,2020-09-19 01:00:00+00:00,101.418278,29.753877,0,0.4
1,Nikola Jokic,15432433,C,16500,Recent,DEN,2020-09-19 01:00:00+00:00,84.768098,40.811629,0,0.4
2,Anthony Davis,15432434,PF/C,16200,Breaking,LAL,2020-09-19 01:00:00+00:00,67.536264,32.078738,0,0.4
3,Jamal Murray,15432435,PG/SG,15000,Recent,DEN,2020-09-19 01:00:00+00:00,59.482523,33.648743,0,0.4
4,Will Barton,15432436,PG,13800,None,DEN,2020-09-19 01:00:00+00:00,77.243669,36.438244,0,0.4
...,...,...,...,...,...,...,...,...,...,...,...
65,Avery Bradley,15432427,PG,1000,None,LAL,2020-09-19 01:00:00+00:00,5.754372,2.844129,0,0.4
66,Troy Daniels,15432428,SG,1000,None,DEN,2020-09-19 01:00:00+00:00,4.995945,1.834112,0,0.4
67,Quinn Cook,15432429,PG,1000,None,LAL,2020-09-19 01:00:00+00:00,5.361482,2.382547,0,0.4
68,Jared Dudley,15432430,PF,1000,None,LAL,2020-09-19 01:00:00+00:00,4.439659,2.639839,0,0.4


# Now let's work with a historical dataframe 
Data is from 2020/01/02

In [7]:
df_players = pd.read_csv('../data/raw/test_20200102.csv')
df_players['Time'] = pd.to_datetime(df_players['Time'])
df_players = df_players[cols]
df_players

,Name,ID,Position,Salary,Status,TeamAbbrev,Time,projections,std,min_exp,max_exp
0,Gary Harris,14066354,SG,4600,recent,MIL,2020-09-10 23:00:00+00:00,29.62,4.462711,0,0.4
1,Darius Garland,14066387,PG,4400,recent,HOU,2020-09-10 23:00:00+00:00,26.53,4.750068,0,0.4
2,Collin Sexton,14066177,PG/SG,5900,recent,HOU,2020-09-10 23:00:00+00:00,34.33,7.762624,0,0.4
3,Kevin Porter,14066428,SG/SF,4200,recent,MIL,2020-09-10 23:00:00+00:00,23.80,5.999999,0,0.4
4,Aaron Holiday,14066210,PG/SG,5700,recent,HOU,2020-09-10 23:00:00+00:00,31.38,5.332358,0,0.4
...,...,...,...,...,...,...,...,...,...,...,...
212,Patrick Patterson,14066669,PF/C,3200,recent,HOU,2020-09-10 23:00:00+00:00,5.28,1.183974,0,0.4
213,Louis King,14066970,SF/PF,3000,recent,MIL,2020-09-10 23:00:00+00:00,4.87,0.616341,0,0.4
214,Jerome Robinson,14066841,SG,3000,recent,HOU,2020-09-10 23:00:00+00:00,4.05,0.474432,0,0.4
215,Johnathan Motley,14066954,PF/C,3000,recent,MIA,2020-09-10 23:00:00+00:00,3.84,0.583016,0,0.4


# Let's do a simple lineup optimization
This is like the original optimizer. Provide the number of desired lineups and the dataframe.
Note that order=1 is an optional argument, but defaults to unordered (0)
time_limit=10.0 by default, but is also an optional argument.

This first step creates the optimization model.

In [11]:
opt = LineupOptimizer(df_players, n_lineups=10, verbose=True)

## opt.solve() solves the optimization problem

In [12]:
opt.solve()

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10
Tried aggregator 1 time.
MIP Presolve eliminated 129 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 209 rows, 990 columns, and 5980 nonzeros.
Reduced MIP has 990 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.13 ticks)
Found incumbent of value 2155.700000 after 0.03 sec. (8.23 ticks)
Probing time = 0.00 sec. (0.29 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 209 rows, 990 columns, and 5980 nonzeros.
Reduced MIP has 990 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.11 ticks)
Probing time = 0.00 sec. (0.29 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.02 sec. (7.12 ticks)

        Nodes 

array([[ 1., -0.,  1.,  0.,  1.,  0., -0.,  1., -0.,  0.],
       [-0.,  0.,  1.,  0., -0.,  1.,  1., -0.,  1., -0.],
       [-0.,  1.,  0.,  1., -0.,  1., -0., -0.,  1., -0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.,  1.,  0., -0.,  1.],
       [ 1.,  0.,  1.,  0., -0.,  1.,  1., -0.,  0.,  0.],
       [-0.,  1., -0.,  0.,  1.,  0., -0.,  1., -0.,  1.],
       [-0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.],
       [ 0.,  1., -0.,  1.,  1.,  1., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -0.,  1.,  1., -0.,  1., -0., -0.,  1., -0.],
       [ 1., -0., -0.,  0., -0.,  0.,  1.,  1.,  1.,  0.],
       [ 1.,  0.,  0.,  1.,  1.,  0.,  1., -0.,  0.,  0.],
       [ 1., -0., -0.,  1., -0.,  0., -0., -0.,  1.,  1.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.],
       [-0.,  1., -0.,  0., -0.,  1.,  1., -0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.

## Now let's sort the optimal lineups

In [13]:
opt.sort_lineups()

6 Infeasible lineups dropped


,PG,SG,SF,PF,C,G,F,UTIL
0,Darius Garland,Collin Sexton,Kevin Porter,LaMarcus Aldridge,Ivica Zubac,Zach LaVine,Jimmy Butler,Shai Gilgeous-Alexander
1,Aaron Holiday,Kevin Porter,Buddy Hield,Bojan Bogdanovic,Ivica Zubac,Donovan Mitchell,LaMarcus Aldridge,Shai Gilgeous-Alexander
2,Spencer Dinwiddie,Gary Harris,Kevin Porter,Bojan Bogdanovic,Ivica Zubac,Jimmy Butler,Marquese Chriss,Domantas Sabonis
3,De'Aaron Fox,Gary Harris,Dillon Brooks,Bojan Bogdanovic,Ivica Zubac,Buddy Hield,Jimmy Butler,DeMar DeRozan


# Lineup Generator and Optimizer
Now we'll use a more complex lineup generation --> monte carlo simulation then select the lineups that have the highest likelihood of being a high scoring team.

<br> The default values are:
<br> var_multiple = 0.5 (we weight lineup projected variance at half the importance of the projected mean)
<br> drop_fraction = 0.5 (half the players are discarded each iteration of generation)
<br> time_limit = 1 (1 second is allowed for each optimization iteration)
<br> duplicates_lim = 100 (once 100 duplicate lineups have been generated, it kills the generator)

## First, instantiate the generator:

In [14]:
gen = LineupGenerator(df_players, n_lineups_to_optimize=150, batch_size=20,
var_multiple=0.5, drop_fraction=0.5, time_limit=1, verbose=False)

## Now run the generator by giving a target number of generated lineups:

In [17]:
# gen.generate(10000)
gen.df_lineups = pd.read_csv('../data/raw/test_lineups.csv', index_col=0)

## Now visualize the player distribution:

In [18]:
players = gen.get_player_distribution(gen.df_lineups)
players[:50]

Ivica Zubac                0.3960
Bojan Bogdanovic           0.3820
DeMar DeRozan              0.3470
Jimmy Butler               0.3395
Spencer Dinwiddie          0.3065
LaMarcus Aldridge          0.3040
Aaron Holiday              0.2960
Harrison Barnes            0.2955
Robert Covington           0.2915
Buddy Hield                0.2905
Collin Sexton              0.2905
Dillon Brooks              0.2855
Darius Garland             0.2845
De'Aaron Fox               0.2735
Marquese Chriss            0.2730
Donovan Mitchell           0.2650
Zach LaVine                0.2535
Jalen Brunson              0.2520
Domantas Sabonis           0.2510
Gary Harris                0.2490
Kevin Porter               0.2365
Shai Gilgeous-Alexander    0.2065
Bam Adebayo                0.1885
Cedi Osman                 0.1520
Richaun Holmes             0.1500
Kyle Lowry                 0.1430
Tristan Thompson           0.1035
Bismack Biyombo            0.0855
Jarrett Culver             0.0850
Shabazz Napier

## Now run the Monte Carlo Simulation

In [19]:
gen.get_lineup_score_dists()

,PG,SG,SF,PF,C,G,F,UTIL,mean,std
0,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Richaun Holmes,Aaron Holiday,Paul George,Kyle Lowry,264.54,17.620039
1,Jalen Brunson,Gary Harris,Kevin Porter,Kawhi Leonard,Marquese Chriss,Aaron Holiday,Paul George,Kyle Lowry,262.63,15.017997
2,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Marquese Chriss,Kyle Lowry,Richaun Holmes,Kawhi Leonard,264.81,17.261263
3,Jalen Brunson,Spencer Dinwiddie,Harrison Barnes,Marquese Chriss,Richaun Holmes,Collin Sexton,Paul George,Aaron Holiday,270.06,20.851468
4,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Richaun Holmes,Shabazz Napier,Paul George,Kyle Lowry,263.37,17.406462
...,...,...,...,...,...,...,...,...,...,...
1995,Darius Garland,Gary Harris,Jimmy Butler,Harrison Barnes,Marquese Chriss,Spencer Dinwiddie,Robert Covington,Donovan Mitchell,279.25,20.311315
1996,Spencer Dinwiddie,Gary Harris,Kevin Porter,Marquese Chriss,Bam Adebayo,Tim Frazier,Jimmy Butler,De'Aaron Fox,277.49,19.496710
1997,Darius Garland,Gary Harris,Kevin Porter,Domantas Sabonis,Bismack Biyombo,De'Aaron Fox,Jimmy Butler,Donovan Mitchell,273.53,17.789721
1998,De'Aaron Fox,Kevin Porter,Dillon Brooks,Bojan Bogdanovic,Bam Adebayo,Buddy Hield,Robert Covington,Shai Gilgeous-Alexander,267.74,18.799925


## Now find the optimal set of lineups:

In [20]:
df_optimal = gen.enforce_exposures()

# Now build the sheet:

In [26]:
sheet_name = 'test_DKEntries2.csv'
entries = EntriesHandler('../data/raw/' + sheet_name, gen.df, read_lineups=True)

In [27]:
entries.add_lineups_to_entries(df_optimal)

In [28]:
entries.write_entries_to_csv()

## Visualize the final sheet

In [29]:
pd.read_csv('../data/raw/test_DKEntries2_v2.csv')

,Entry ID,Contest Name,Contest ID,Entry Fee,PG,SG,SF,PF,C,G,F,UTIL
0,1809803072,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14066354,14066428,14065971,14066260,14066210,14065986,14066006
1,1809803073,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14065998,14066428,14066304,14066260,14066210,14065971,14066006
2,1809803074,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14065998,14066304,14066260,14066065,14066177,14065986,14066186
3,1809803075,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066058,14066354,14066428,14066137,14066019,14066033,14066340,14066061
4,1809803076,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14065998,14066340,14066260,14065994,14066492,14065989,14066053
...,...,...,...,...,...,...,...,...,...,...,...,...
145,1809803217,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14066015,14066428,14066260,14066415,14066033,14065989,14066006
146,1809803218,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14065998,14066088,14066137,14066415,14066177,14065989,14066228
147,1809803219,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14065998,14066428,14066137,14066415,14066033,14066088,14066019
148,1809803220,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14065998,14065989,14066260,14066415,14066177,14066228,14066058
